In [ ]:
import numpy as np
import json
import csv
import os
import math
import tkinter as tk
from tkinter import messagebox, ttk
from typing import Optional
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from datetime import datetime

# -----------------------------
# CONSTANTS
# -----------------------------
TOTAL_SLOTS   = 5
RATE_PER_HOUR = 50
FOLDER_NAME   = "ParkingSystem"
BASE_PATH     = os.path.join(os.getcwd(), FOLDER_NAME)
os.makedirs(BASE_PATH, exist_ok=True)

PARKING_FILE = os.path.join(BASE_PATH, "parking_data.json")
DATASET_JSON = os.path.join(BASE_PATH, "parking_dataset.json")
DATASET_CSV  = os.path.join(BASE_PATH, "parking_dataset.csv")

# -----------------------------
# HELPERS
# -----------------------------
def empty_slot(distance: float) -> dict:
    return {
        "status": 0,
        "vehicle": None,
        "entry_time": None,
        "distance": distance
    }

# -----------------------------
# INITIALIZE STORAGE
# -----------------------------
if os.path.isfile(PARKING_FILE):
    with open(PARKING_FILE) as f:
        parking_slots = json.load(f)
else:
    parking_slots = [empty_slot(round((i + 1) * 5.0, 1)) for i in range(TOTAL_SLOTS)]
    with open(PARKING_FILE, "w") as f:
        json.dump(parking_slots, f, indent=2)

if os.path.isfile(DATASET_JSON):
    with open(DATASET_JSON) as f:
        dataset = json.load(f)
else:
    dataset = []
    with open(DATASET_JSON, "w") as f:
        json.dump(dataset, f, indent=2)

# -----------------------------
# SAVE DATA
# -----------------------------
def save_dataset():
    with open(PARKING_FILE, "w") as f:
        json.dump(parking_slots, f, indent=2)

    with open(DATASET_JSON, "w") as f:
        json.dump(dataset, f, indent=2)

    with open(DATASET_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=[
                "Slot_Number", "Vehicle_Name", "Distance",
                "Entry_Time", "Exit_Time", "Parking_Fee"
            ]
        )
        writer.writeheader()
        for d in dataset:
            writer.writerow({
                "Slot_Number": d["slot_number"],
                "Vehicle_Name": d["vehicle"],
                "Distance": d["distance"],
                "Entry_Time": d["entry_time"],
                "Exit_Time": d["exit_time"],
                "Parking_Fee": d["fee"]
            })

# -----------------------------
# MACHINE LEARNING
# -----------------------------
ml_model = None
accuracy_score = 0.0

def train_ml():
    global ml_model, accuracy_score

    X = [[float(d["distance"])] for d in dataset if d.get("duration_minutes") is not None]
    y = [float(d["duration_minutes"]) for d in dataset if d.get("duration_minutes") is not None]

    if len(X) >= 2:
        ml_model = LinearRegression()
        ml_model.fit(X, y)
        preds = ml_model.predict(X)
        accuracy_score = round(r2_score(y, preds) * 100, 2)
    else:
        ml_model = None
        accuracy_score = 0.0

train_ml()

# -----------------------------
# PREDICTION FUNCTION
# -----------------------------
def predict_remaining_time(slot: dict) -> int:
    if slot["status"] == 0 or slot["entry_time"] is None:
        return 0

    entry_time = datetime.strptime(slot["entry_time"], "%Y-%m-%d %H:%M:%S")
    elapsed = (datetime.now() - entry_time).total_seconds() / 60

    if ml_model is not None:
        predicted_total = ml_model.predict([[float(slot["distance"])]])[0]
    else:
        durations = [d["duration_minutes"] for d in dataset if d.get("duration_minutes")]
        predicted_total = np.mean(durations) if durations else 30

    return max(int(predicted_total - elapsed), 0)

# -----------------------------
# PARK / RELEASE
# -----------------------------
def park_vehicle(plate: str) -> Optional[int]:
    empty = [(s["distance"], i) for i, s in enumerate(parking_slots) if s["status"] == 0]
    if not empty:
        return None

    _, idx = min(empty)
    parking_slots[idx].update({
        "status": 1,
        "vehicle": plate,
        "entry_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })
    save_dataset()
    return idx

def release_vehicle(idx: int):
    slot = parking_slots[idx]
    if slot["status"] == 0:
        return None

    entry = datetime.strptime(slot["entry_time"], "%Y-%m-%d %H:%M:%S")
    exit_time = datetime.now()
    duration = int((exit_time - entry).total_seconds() / 60)
    fee = math.ceil(duration / 60) * RATE_PER_HOUR

    record = {
        "slot_number": idx + 1,
        "vehicle": slot["vehicle"],
        "distance": slot["distance"],
        "entry_time": entry.strftime("%Y-%m-%d %H:%M:%S"),
        "exit_time": exit_time.strftime("%Y-%m-%d %H:%M:%S"),
        "duration_minutes": duration,
        "fee": fee
    }

    dataset.append(record)
    parking_slots[idx] = empty_slot(slot["distance"])
    save_dataset()
    train_ml()
    return record

# -----------------------------
# GUI
# -----------------------------
gui = tk.Tk()
gui.title("Intelligent Parking Management System")
gui.geometry("640x560")
gui.resizable(False, False)

status_labels = []
for i in range(TOTAL_SLOTS):
    frame = tk.LabelFrame(gui, text=f"Slot {i+1}")
    frame.place(x=30, y=40*i+40, width=270, height=55)
    lbl = tk.Label(frame, anchor="w")
    lbl.pack(fill="both")
    status_labels.append(lbl)

tk.Label(gui, text="Vehicle Name").place(x=330, y=10)
vehicle_field = tk.Entry(gui, width=25)
vehicle_field.place(x=330, y=30)

tk.Label(gui, text="Slot #").place(x=480, y=10)
slot_field = tk.Entry(gui, width=10)
slot_field.place(x=480, y=30)

park_btn = tk.Button(gui, text="Park")
remove_btn = tk.Button(gui, text="Remove")
history_btn = tk.Button(gui, text="History")

park_btn.place(x=330, y=70, width=130)
remove_btn.place(x=480, y=70, width=130)
history_btn.place(x=330, y=110, width=280)

pred_label = tk.Label(gui, fg="blue")
pred_label.place(x=330, y=150, width=280)

acc_label = tk.Label(gui, fg="green")
acc_label.place(x=330, y=170, width=280)

# -----------------------------
# GUI LOGIC
# -----------------------------
def redraw():
    next_free_times = []

    for i, lbl in enumerate(status_labels):
        s = parking_slots[i]
        if s["status"] == 0:
            lbl.config(text="EMPTY", bg="lightgreen")
        else:
            rem = predict_remaining_time(s)
            next_free_times.append(rem)
            lbl.config(text=f"{s['vehicle']}", bg="salmon")  # only vehicle name

    if next_free_times:
        pred_label.config(text=f"Next slot frees ≈ {min(next_free_times)} min")
    else:
        pred_label.config(text="Slots available now")

    acc_label.config(
        text=f"Model accuracy: {accuracy_score}%" if accuracy_score > 0 else "Not enough data"
    )

    gui.after(2000, redraw)

def park_click():
    plate = vehicle_field.get().strip().upper()
    if not plate:
        messagebox.showerror("Error", "Enter vehicle name")
        return
    idx = park_vehicle(plate)
    if idx is None:
        messagebox.showwarning("Full", pred_label.cget("text"))
    else:
        messagebox.showinfo("Parked", f"Vehicle '{plate}' parked in slot {idx+1}")
    vehicle_field.delete(0, tk.END)

def remove_click():
    if not slot_field.get().isdigit():
        messagebox.showerror("Error", "Enter valid slot number")
        return
    idx = int(slot_field.get()) - 1
    if idx < 0 or idx >= TOTAL_SLOTS:
        messagebox.showerror("Error", "Invalid slot")
        return
    record = release_vehicle(idx)
    if record:
        messagebox.showinfo("Released", f"Vehicle '{record['vehicle']}' released | Fee: Rs {record['fee']}")
    else:
        messagebox.showwarning("Empty", "Slot already empty")
    slot_field.delete(0, tk.END)

def open_history():
    win = tk.Toplevel(gui)
    win.title("Parking History")
    win.geometry("820x420")
    cols = ["Slot", "Vehicle", "Distance", "Entry", "Exit", "Fee"]
    tv = ttk.Treeview(win, columns=cols, show="headings")
    for c in cols:
        tv.heading(c, text=c)
    tv.pack(fill="both", expand=True)

    for d in dataset:
        tv.insert("", "end", values=[
            d["slot_number"], d["vehicle"], d["distance"],
            d["entry_time"], d["exit_time"], d["fee"]
        ])

park_btn.config(command=park_click)
remove_btn.config(command=remove_click)
history_btn.config(command=open_history)

redraw()
gui.mainloop()
